# Library import

In [2]:
import openpyxl as op
import pandas as pd
import os
import numpy as np
import win32com.client as win32

import matplotlib.pyplot as plt
parameters = {'axes.labelsize': 17,
              'xtick.labelsize': 16,
              'ytick.labelsize': 16,
          'axes.titlesize': 20}
plt.rcParams.update(parameters)
plt.rcParams['font.family'] = 'arial'

In [11]:
basepath = os.getcwd()
def SaveFile(filename):
    excel = win32.Dispatch("Excel.Application") 
    wb = excel.Workbooks.Open(r"{}\{}".format(basepath, filename))
    # wb = excel.Workbooks.Open(r"C:\Users\안나현\Documents\GitHub\TeamEnv\TEA\{}".format(filename))
    excel.Visible = False
    wb.Save()
    excel.Quit()

# Plastric pyrolysis

## TEA functions

In [12]:
def InputPyroValue(filename, target_data, country_data):
    
    # TEA file open
    wb = op.load_workbook(filename)
    capex_opex = wb['Plastic']
    
    # CAPEX
    cells = [f'F{jj}' for jj in range(5,9)] + [f'F{jj}' for jj in range(10,15)]
    blocks_input = ['HDPE[kg/hr]', 'LDPE[kg/hr]', 'PP[kg/hr]', 'PS[kg/hr]',
               'HEATER_HX_AREAC', 'RPLUGIN2', 'RPYROIN', 'COOLER_HX_AREAC', 'S1']     # RPLUGIN, RPYROOUT ==> 열교환면적 데이터로 수정
    
    for _blck, _cell in zip(blocks_input, cells):
        capex_opex[_cell] = target_data[_blck]
        
    # OPEX
    cells = [f'C{jj}' for jj in range(23,27)] + [f'C{jj}' for jj in range(28,32)]
    block_hd = ['HDPDCOM_HD', 'LPDECOM_HD', 'PPDECOM_HD', 'PSDECOM_HD',
              'HEATER_HD', 'RPLUG_HD', 'RPYRO_HD', 'COOLER_HX_DUTY']
    
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
        
    # Revenue
    revenue_sheet = wb['Revenue']
    cells = [f'C{jj}' for jj in range(4,17)]
    liq_comp = [revenue_sheet[f'B{ii}'].value+'_LIQ' for ii in range(4,17)]   
    for _comp, _cell in zip(liq_comp, cells):
        revenue_sheet[_cell] = target_data[_comp]
    revenue_sheet['C17'] = target_data['CH4_VAP']   # NG판매시 고려
        
    # Parameter
    cells = [f'O{jj}' for jj in range(5,17)]
    for _par, _cell in zip(country_data, cells):
        revenue_sheet[_cell] = _par
        
    wb.save(filename=filename)
    wb.close()   

def LoadPyroResults(filename):
    re_load = op.load_workbook(filename, data_only=True)
    tea_res = re_load['Plastic EAC&TPC']
    eac = tea_res['E34'].value
    tpc = tea_res['K25'].value
    
    rvn_sht = re_load['Revenue']
    rvn_SMR = rvn_sht['D19'].value
    rvn_NG = rvn_sht['C19'].value
    
    return eac, tpc, rvn_SMR,rvn_NG

## Load aspen data

In [14]:
data = pd.read_csv('../data/1. Aspen_results/Results_pyrolysis_python_v2.csv')
tea_param = pd.read_csv('../data/Revenue_params.csv')
data.head()

# tea_param.columns

,Country,HDPE[kg/hr],LDPE[kg/hr],PP[kg/hr],PS[kg/hr],HEATER_T,HDPDCOM_HD,LPDECOM_HD,PPDECOM_HD,PSDECOM_HD,...,C6H6O_LIQ,N2_LIQ,CL2_LIQ,S_LIQ,NRP_LIQ,ASH_LIQ,HDPE_LIQ,PP_LIQ,PS_LIQ,LDPE_LIQ
0,Brazil,10,20,8,8,500,2396.45547,4751.80836,1980.26732,746.788816,...,0,1.570000e-06,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,10,20,8,8,550,2396.45547,4751.80836,1980.26732,746.788816,...,0,1.340000e-06,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,10,20,8,8,600,2396.45547,4751.80836,1980.26732,746.788816,...,0,1.090000e-06,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,10,20,8,8,650,2396.45547,4751.80836,1980.26732,746.788816,...,0,9.330000e-07,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Brazil,10,20,8,8,700,2396.45547,4751.80836,1980.26732,746.788816,...,0,8.250000e-07,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Analyse economics

In [15]:
tea_file = 'TEA_plastic_pyro_v3.xlsx'

tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    _country = target_data['Country']
    TEA_PAR = tea_param[_country]
    TEA_PAR[0] = 12
    
    InputPyroValue(tea_file, target_data, TEA_PAR)
    SaveFile(tea_file)
    EAC, TPC, RVN_SMR, RVN_NG = LoadPyroResults(tea_file)
    tea_res.append([EAC, TPC, RVN_SMR, RVN_NG])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_289456\1204473164.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEA_PAR[0] = 12


## Save results

In [16]:
data[['pyro_EAC', 'pyro_TPC', 'pyro_REV_SMR', 'pyro_REV_NG']] = tea_res
data.to_csv('../results/2. TEA_results/Results_TEA_pyrolysis_v4.csv')

# SMR process

## TEA functions

In [21]:
def InputSMRValue(filename, target_data, country_data):
    
    # TEA file open
    wb = op.load_workbook(filename)
    capex_opex = wb['SMR']
    
    # CAPEX
    tmp_cells = [f'F{jj}' for jj in range(5,21)]
    dump = ['F7','F12','F17','F18',]
    cells = [i for i in tmp_cells if i not in dump]
    
    blocks_input = ['COMP_BRAKE_POWER', 'HEAT1_HX_AREAC2', 'SMR_BAL_MASI_TFL', 
                    'PUMP_BRAKE_POWER', 'HX1_HX_AREAC2', 'HX2_HX_AREAC2',
                    'WGS_BAL_MASI_TFL','COOL1_HX_AREAC2','DRUM_BAL_MASI_TFL', 
                    'PSA_BAL_MOLI_TFL', 'HEAT2_HX_AREAC2', 'VSA_BAL_MASI_TFL']
    
    for _blck, _cell in zip(blocks_input, cells):
        capex_opex[_cell] = target_data[_blck]
        
    # OPEX
    tmp_cells = [f'C{jj}' if jj == 30 or jj == 34 else f'D{jj}' for jj in range(30,45)]
    dump = ['D32','D34','D37','D38','D40','D42','D43',]
    cells = [i for i in tmp_cells if i not in dump]

    block_hd = ['COMP_BRAKE_POWER', 'HEAT1_HX_DUTY', 'SMR_QCALC',
                'PUMP_BRAKE_POWER','HX1_HX_DUTY','HX2_HX_DUTY',
                'COOL1_HX_DUTY', 'PSA_QCALC', 'HEAT2_HX_DUTY']
    
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
    
    # Revenue
    revenue_sheet = wb['Revenue']
    revenue_sheet['C3'] = target_data['H2']
    
    # Parameter
    cells = [f'O{jj}' for jj in range(5,17)]
    for _par, _cell in zip(country_data, cells):
        revenue_sheet[_cell] = _par
        
    wb.save(filename=filename)
    wb.close()   

def LoadSMRResults(filename):
    re_load = op.load_workbook(filename, data_only=True)
    tea_res = re_load['SMR EAC&TPC']
    eac = tea_res['E34'].value
    tpc = tea_res['K25'].value
    
    rvn_sht = re_load['Revenue']
    rvn = rvn_sht['E3'].value
    return eac, tpc, rvn

## Load Aspen data

In [19]:
data = pd.read_csv('../data/1. Aspen_results/Results_SMR_v4_hx.csv')
tea_param = pd.read_csv('../data/Revenue_params.csv')
data.head()

,Country,PUMP_BRAKE_POWER,COMP_BRAKE_POWER,HEAT1_HX_AREAC2,HEAT1_HX_DUTY,HEAT2_HX_AREAC2,HEAT2_HX_DUTY,COOL1_HX_AREAC2,COOL1_HX_DUTY,HX1_HX_AREAC2,...,S5,S6,S7,S8,S9,S9-1,S9-2,CO,CO2.1,H2.1
0,Brazil,0.030314,1.155510,0.011094,79.528181,0.009090,10.871952,0.255057,-1431.32044,0.031016,...,14.584160,18.912961,18.912961,18.912961,18.912961,18.912961,0,0.280104,8.186416,0.060476
1,Brazil,0.031784,1.211545,0.011636,83.384796,0.009532,11.399248,0.268021,-1500.69839,0.032522,...,15.291400,19.830120,19.830120,19.830120,19.830120,19.830120,0,0.280104,8.583495,0.060476
2,Brazil,0.032172,1.226319,0.011780,84.401622,0.009651,11.541509,0.271260,-1517.61014,0.032987,...,15.477869,20.071936,20.071936,20.071936,20.071936,20.071936,0,0.280104,8.692009,0.060476
3,Brazil,0.032158,1.225779,0.011774,84.364451,0.009646,11.536426,0.271135,-1516.94175,0.032972,...,15.471053,20.063096,20.063096,20.063096,20.063096,20.063096,0,0.280104,8.688181,0.060476
4,Brazil,0.031864,1.214584,0.011666,83.593924,0.009558,11.431059,0.268538,-1503.08709,0.032671,...,15.329751,19.879853,19.879853,19.879853,19.879853,19.879853,0,0.280104,8.608829,0.060476


## Analyse economics

In [22]:
tea_file = 'TEA_SMR process_v2.xlsx'

tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    _country = target_data['Country']
    TEA_PAR = tea_param[_country]
    # TEA_PAR[0] = 4.68
    
    InputSMRValue(tea_file, target_data, TEA_PAR)
    SaveFile(tea_file)
    EAC, TPC, Rvn = LoadSMRResults(tea_file)
    tea_res.append([EAC, TPC, Rvn])

## Save results

In [23]:
data[['SMR_EAC', 'SMR_TPC', 'SMR_REV']] = tea_res
data.to_csv('../results/2. TEA_results/Results_TEA_SMR_v3.csv')

# CCU process

In [24]:
def InputSMRCCUValue(filename, target_data, country_data):
    
    # TEA file open
    wb = op.load_workbook(filename)
    capex_opex = wb['SMRCCU']
    
    # CAPEX
    cells = [f'F{jj}' for jj in range(5,13)]    
    blocks_input = ['COMP2_BRAKE_POWER', 'COOL2_HX_AREAC',
                    'COMP3_BRAKE_POWER', 'COOL3_HX_AREAC',
                    'COMP4_BRAKE_POWER', 'COOL4_HX_AREAC',
                    'COMP5_BRAKE_POWER', 'COOL5_HX_AREAC',]
    
    for _blck, _cell in zip(blocks_input, cells):
        capex_opex[_cell] = target_data[_blck]
        
    # OPEX
    cells = [f'C{jj}' for jj in range(18,25,2)]
    block_hd = ['COMP2_BRAKE_POWER','COMP3_BRAKE_POWER',
                'COMP4_BRAKE_POWER','COMP5_BRAKE_POWER' ]
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
        
    cells = [f'D{jj}' for jj in range(19,26,2)]
    block_hd = ['COOL2_HX_DUTY','COOL3_HX_DUTY',
                'COOL4_HX_DUTY','COOL5_HX_DUTY' ]
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
    
    # Revenue
    revenue_sheet = wb['Revenue']
    revenue_sheet['C3'] = target_data['CO2']
    
    # Parameter
    cells = [f'O{jj}' for jj in range(5,17)]
    for _par, _cell in zip(country_data, cells):
        revenue_sheet[_cell] = _par
        
    wb.save(filename=filename)
    wb.close()   

def LoadSMRCCUResults(filename):
    re_load = op.load_workbook(filename, data_only=True)
    tea_res = re_load['SMRCCU EAC&TPC']
    eac = tea_res['E34'].value
    tpc = tea_res['I25'].value
    
    rvn_sht = re_load['Revenue']
    rvn = rvn_sht['E3'].value
    return eac, tpc, rvn

## Load aspen data

In [25]:
data = pd.read_csv('../data/1. Aspen_results/Results_SMR_CCU.csv')
tea_param = pd.read_csv('../data/Revenue_params.csv')
data.head()

# tea_param.columns

,Country,HDPE[kg/hr],LDPE[kg/hr],Country.1,PP[kg/hr],PS[kg/hr],HEATER_T,CH4 flowrate,COOL2_HX_AREAC,COOL3_HX_AREAC,...,COOL5_HX_AREAC,COMP2_BRAKE_POWER,COMP3_BRAKE_POWER,COMP4_BRAKE_POWER,COMP5_BRAKE_POWER,COOL2_HX_DUTY,COOL3_HX_DUTY,COOL4_HX_DUTY,COOL5_HX_DUTY,CO2
0,Brazil,10,20,Brazil,8,8,500,4.328800,0.010097,0.010807,...,0.014141,0.243337,0.237141,0.216954,0.064134,56.662032,60.646043,60.646043,79.354837,8.186416
1,Brazil,10,20,Brazil,8,8,550,4.538720,0.010610,0.011356,...,0.014860,0.255137,0.248640,0.227475,0.067244,59.408514,63.585635,63.585635,83.201269,8.583495
2,Brazil,10,20,Brazil,8,8,600,4.594066,0.010738,0.011493,...,0.015039,0.258249,0.251672,0.230249,0.068064,60.078004,64.302198,64.302198,84.138885,8.692009
3,Brazil,10,20,Brazil,8,8,650,4.592043,0.010733,0.011488,...,0.015032,0.258135,0.251562,0.230147,0.068034,60.051544,64.273878,64.273878,84.101828,8.688181
4,Brazil,10,20,Brazil,8,8,700,4.550103,0.010631,0.011378,...,0.014888,0.255777,0.249264,0.228045,0.067413,59.503076,63.686846,63.686846,83.333702,8.608829


In [26]:
tea_file = 'TEA_SMRCCU.xlsx'

tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    _country = target_data['Country']
    TEA_PAR = tea_param[_country]
    
    InputSMRCCUValue(tea_file, target_data, TEA_PAR)
    SaveFile(tea_file)
    EAC, TPC, Rvn = LoadSMRCCUResults(tea_file)
    tea_res.append([EAC, TPC, Rvn])

## Save results

In [27]:
data[['SMRCCU_EAC', 'SMRCCU_TPC', 'SMRCCU_REV']] = tea_res
data.to_csv('../results/2. TEA_results/Results_TEA_SMRCCU.csv')

# WFGD process

## TEA functions

In [28]:
def InputWFGDValue(filename, target_data, country_data):
    
    # TEA file open
    wb = op.load_workbook(filename)
    capex_opex = wb['Desulf']
    
    # CAPEX
    cells = [f'F{jj}' for jj in range(5,10)]
    blocks_input = ['LIME-DS_BAL_MASI_TFL', 'SCRUBBER_TOT_VOL', 
                    'GYP-CR_TOT_VOL', 'GAS-SP_BAL_MASI_TFL', 'GYP-SP_BAL_MASI_TFL'] 
    factor = [0.264172 if blc[-3:]=='VOL' else 1 for blc in blocks_input]
    
    for _blck, _cell, _fact in zip(blocks_input, cells, factor):
        capex_opex[_cell] = target_data[_blck] * _fact
                                                #0.264172gal/1L
        
    # OPEX
    cells = [f'C{jj}' for jj in range(18,23)]
    block_hd = ['LIME-DS_QCALC', 'SCRUBBER_QCALC', 'GYP-CR_QCALC',
                'GAS-SP_QCALC', 'GYP-SP_QCALC']
    
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
        
    # Revenue
    revenue_sheet = wb['Desulf revenue']
    revenue_sheet['C3'] = target_data['gypsum production']
    
    # Parameter
    cells = [f'O{jj}' for jj in range(5,17)]
    for _par, _cell in zip(country_data, cells):
        revenue_sheet[_cell] = _par
        
    wb.save(filename=filename)
    wb.close()   

def LoadWFGDResults(filename):
    re_load = op.load_workbook(filename, data_only=True)
    tea_res = re_load['Desulf EAC&TPC']
    eac = tea_res['E34'].value
    tpc = tea_res['K25'].value
    
    rvn_sht = re_load['Desulf revenue']
    rvn = rvn_sht['E3'].value
    return eac, tpc, rvn

## Load aspen data

In [29]:
data = pd.read_csv('../data/1. Aspen_results/Results_WFGD_v4.csv')
data.head()

,Country,HDPE[kg/hr],LDPE[kg/hr],PP[kg/hr],PS[kg/hr],HEATER_T,LIME-DS_BAL_MASI_TFL,LIME-DS_QCALC,SCRUBBER_TOT_VOL,SCRUBBER_QCALC,...,CO2,SO2,O2,CALCI-01,CALCI-02,N2,NA2O,AR,gypsum production,Required WOS(kg/h)
0,Brazil,10,20,8,8,500,0.434800,0.304375,7468.70759,54.120590,...,0.464466,0.000135,0.376212,3.150000e-85,1.050000e-79,2.611933,1.590000e-81,0.001481,0.026285,0.016723
1,Brazil,10,20,8,8,550,1.686004,1.180262,28958.80420,209.579343,...,1.801037,0.000138,1.458722,3.230000e-87,4.120000e-79,10.128172,6.150000e-81,0.005744,0.103443,0.064846
2,Brazil,10,20,8,8,600,2.979453,2.085722,51171.63430,370.275128,...,3.182736,0.000125,2.577778,6.660000e-87,7.290000e-79,17.898184,1.090000e-80,0.010150,0.183111,0.114594
3,Brazil,10,20,8,8,650,4.315654,3.021109,74122.10980,536.348384,...,4.610106,0.000202,3.733845,2.230000e-87,1.050000e-78,25.925021,1.580000e-80,0.014702,0.264011,0.165987
4,Brazil,10,20,8,8,700,5.695030,3.986721,97809.37230,707.654579,...,6.083600,0.000099,4.927222,1.340000e-84,1.380000e-78,34.211220,2.080000e-80,0.019401,0.346677,0.219040


## Analyse economics

In [30]:
tea_file = 'TEA_WSS desulfurization.xlsx'

tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    _country = target_data['Country']
    TEA_PAR = tea_param[_country]
    TEA_PAR[0] = TEA_PAR[0]*1.5
    
    InputWFGDValue(tea_file, target_data,TEA_PAR)
    SaveFile(tea_file)
    EAC, TPC, Rvn = LoadWFGDResults(tea_file)
    tea_res.append([EAC, TPC, Rvn])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_289456\101184855.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TEA_PAR[0] = TEA_PAR[0]*1.5


In [31]:
data[['WFGD_EAC', 'WFGD_TPC', 'WFGD_REV']] = tea_res
data.to_csv('../results/2. TEA_results/Results_TEA_WFGD_v5+50.csv')

# Total results

In [33]:
data = pd.read_csv('../data/1. Aspend_results/TEA_results_each_12.csv')
data.head()

,Country,HDPE[kg/hr],LDPE[kg/hr],PP[kg/hr],PS[kg/hr],HEATER_T,pyro_EAC,pyro_TPC,pyro_REV_SMR,pyro_REV_NG,SMR_EAC,SMR_TPC,SMR_REV,SMRCCU_EAC,SMRCCU_TPC,SMRCCU_REV,WFGD_EAC,WFGD_TPC,WFGD_REV
0,Brazil,10,20,8,8,500,75975.13531,320790.9109,215973.2838,243100.2509,62850.35948,181575.2864,158254.7575,2672.702932,7039.418655,1670.028935,30094.72199,5877.392349,111.710107
1,Brazil,10,20,8,8,550,75993.22094,322239.9877,214055.6847,242498.1384,64852.21697,190217.1639,165929.4894,2766.536441,7375.118241,1751.032919,67829.35882,15169.299920,439.633787
2,Brazil,10,20,8,8,600,75998.90254,336280.3360,208099.3531,236888.6447,65377.90373,192471.8294,167969.0896,2790.423323,7459.255153,1773.169769,95433.73850,22998.463790,778.223318
3,Brazil,10,20,8,8,650,76008.55567,351934.5090,204195.8507,232972.4636,65358.83216,192388.5781,167895.1181,2789.525973,7456.021623,1772.388967,119179.89360,30361.083480,1122.046453
4,Brazil,10,20,8,8,700,76017.89517,369061.4829,201597.6962,230111.4831,64962.85471,190662.7176,166361.6756,2770.905498,7388.991027,1756.201026,140745.48810,37508.865790,1473.378117


In [34]:
filename = 'TEA_eachPathway.xlsx'

In [35]:

# import win32com.client

def InputTEAValue(filename, target_data):
    alpha = ['B','C','D','E','F','G','H','I','J','K','L','M','N']
    cells = [f'{al}3' for al in alpha]
    
    wb = op.load_workbook(filename)
    sht = wb['Sheet1']
    for _val, _cell in zip(target_data, cells):
        sht[_cell] = _val
    wb.save(filename=filename)
    wb.close()  

def LoadTEAResults(filename):
   
    re_load = op.load_workbook(filename, data_only=True)
    sht = re_load['Sheet1']
    res=[]
    for ii in range(4,10):
        res_eac = sht[f'U{ii}'].value
        res_tpc = sht[f'V{ii}'].value
        res_rev = sht[f'W{ii}'].value
        res.append([res_eac, res_tpc, res_rev])
    return res

In [36]:
path_name = [[f'Path{i}_EAC', f'Path{i}_TPC',f'Path{i}_REV'] for i in range(1,7)]

filename = 'TEA_eachPathway.xlsx'
tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    targ_val = target_data[-13:]
    InputTEAValue(filename, targ_val)
    SaveFile(filename)
    res = LoadTEAResults(filename)
    tea_res.append(res)

In [37]:
tesetset = np.array(tea_res).reshape( -1,18)
path_name_nd = list(np.array(path_name).flatten())
data[path_name_nd] = tesetset

In [38]:
data.to_csv('../results/2. TEA_results/TotalResults_12.csv')

# TEA